In [1]:
import pandas as pd
import numpy as np
import os
import magic
import json
import random
import traceback

from browsermobproxy import Server
from selenium import webdriver

from util import is_json, parse_har, write_har, get_driver_and_proxy
from crawler import random_walk_page
from feature_gen import gen_feature

In [3]:
load_path = "../data/cs.uchicago.edu/"
fpd = gen_feature(load_path)

In [4]:
fpd.head(5)

,path,size,type
0,diversity/equity-diversity-and-inclusion-edi-c...,103393,text/html
1,diversity/compileher/index.html,93111,text/html
2,diversity/broadening-participation-in-computin...,94474,text/html
3,diversity/grace-hopper-sponsorship/index.html,93669,text/html
4,diversity/women-in-computing/index.html,95205,text/html


In [3]:
# random_walk_page(driver, "https://cs.uchicago.edu", 5)
# har = proxy.har # returns a HAR JSON blob

driver, proxy, server = get_driver_and_proxy(8080)

try:
    random_walk_page(driver, "https://cs.uchicago.edu", 5)
except:
    traceback.print_exc()
finally:
    server.stop()
    driver.quit()

The geckodriver version (0.31.0) detected in PATH at /usr/local/bin/geckodriver might not be compatible with the detected firefox version (119.0); currently, geckodriver 0.33.0 is recommended for firefox 119.*, so it is advised to delete the driver in PATH and retry


Visiting page https://cs.uchicago.edu, 5 clicks remaining
There are 161 clickable links on this page
Visiting page https://cs.uchicago.edu/about/our-building/, 4 clicks remaining
There are 108 clickable links on this page
Visiting page https://cs.uchicago.edu/research/theory/, 3 clicks remaining
There are 160 clickable links on this page
Visiting page https://cs.uchicago.edu/research/scientific-high-performance-computing/, 2 clicks remaining
There are 157 clickable links on this page
Visiting page https://cs.uchicago.edu/news/uchicago-argonne-researchers-will-cultivate-ai-model-gardens-with-3-5m-nsf-grant/, 1 clicks remaining
There are 123 clickable links on this page
Visiting page https://cs.uchicago.edu/news/assistant-professor-chenhao-tan-receives-sloan-research-fellowship/, 0 clicks remaining
There are 123 clickable links on this page


In [18]:
parse_har(har)

,url,start_time,response_code,body_size,rtt
0,https://cs.uchicago.edu/,2023-11-08T02:47:32.391-06:00,200,146844,354
1,https://tracking-protection.cdn.mozilla.net/ad...,2023-11-08T02:47:32.472-06:00,200,58902,92
2,https://tracking-protection.cdn.mozilla.net/so...,2023-11-08T02:47:32.496-06:00,200,2197,19
3,https://tracking-protection.cdn.mozilla.net/an...,2023-11-08T02:47:32.514-06:00,200,10838,14
4,https://tracking-protection.cdn.mozilla.net/co...,2023-11-08T02:47:32.530-06:00,200,15318,19
...,...,...,...,...,...
128,https://yt3.ggpht.com/ytc/APkrFKbaHXxA7cDMONvW...,2023-11-08T02:47:38.901-06:00,200,5289,174
129,https://www.google.com/js/th/87YGUa-CtHzGh2dX4...,2023-11-08T02:47:38.904-06:00,200,14760,110
130,https://www.google-analytics.com/g/collect?v=2...,2023-11-08T02:47:38.922-06:00,204,0,12
131,https://www.youtube.com/generate_204?cWHO2g,2023-11-08T02:47:38.959-06:00,204,0,2
